In [4]:
import numpy as np
import pandas as pd

In [24]:
def excel_sheets_to_dataframes(excel_file_path):

    # Load the Excel file
    excel_data = pd.ExcelFile(excel_file_path)
    
    dataframes = {}

    # Iterate over each sheet
    for sheet_name in excel_data.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

        # # Optionally drop rows and columns that are completely empty
        df.dropna(how='all', axis=0, inplace=True)
        df.dropna(how='all', axis=1, inplace=True)

        sanitized_name = sheet_name.lower().replace(" ", "_")
        print(sanitized_name)

        # Store the DataFrame in the dictionary
        dataframes[sanitized_name] = df

        # Save them as csvs
        df.to_csv("../../data/preprocessed/scoring_indicators/"+sanitized_name+"_2024.csv")
        print("../../data/preprocessed/scoring_indicators/"+sanitized_name+"_2024.csv")
    return dataframes


In [25]:
excel_file = "../../data/raw/scoring_indicators/2024stablecommunities.xlsx"
dfs_dict = excel_sheets_to_dataframes(excel_file)

data_sources
../../data/preprocessed/scoring_indicators/data_sources_2024.csv
percentiles
../../data/preprocessed/scoring_indicators/percentiles_2024.csv
environmental_health_index
../../data/preprocessed/scoring_indicators/environmental_health_index_2024.csv
transit_access_index
../../data/preprocessed/scoring_indicators/transit_access_index_2024.csv
above_poverty_level
../../data/preprocessed/scoring_indicators/above_poverty_level_2024.csv
median_income
../../data/preprocessed/scoring_indicators/median_income_2024.csv
jobs_proximity_index
../../data/preprocessed/scoring_indicators/jobs_proximity_index_2024.csv


In [26]:
print(dfs_dict['environmental_health_index']['2020 Census Tract'].nunique())
print(dfs_dict['transit_access_index']['2020 Census Tract'].nunique())
print(dfs_dict['above_poverty_level']['2020 Census Tract'].nunique())
print(dfs_dict['median_income']['2020 Census Tract'].nunique())
print(dfs_dict['jobs_proximity_index']['2020 Census Tract'].nunique())

2796
2796
2796
2796
2796


In [28]:
# Merge the first two
df_merged = pd.merge(dfs_dict['environmental_health_index'], 
                     dfs_dict['transit_access_index'],
                     on="2020 Census Tract",
                     how="outer")

# Merge the next
df_merged = pd.merge(df_merged, 
                     dfs_dict['above_poverty_level'], 
                     on="2020 Census Tract", 
                     how="outer")

# Merge the next
df_merged = pd.merge(df_merged, 
                     dfs_dict['median_income'], 
                     on="2020 Census Tract", 
                     how="outer")

# Merge the final
df_merged = pd.merge(df_merged, 
                     dfs_dict['jobs_proximity_index'], 
                     on="2020 Census Tract", 
                     how="outer")

In [29]:
df_merged.dtypes

2020 Census Tract                                  int64
Environmental Health Index                       float64
Transit Access Index                              object
Percent of Population Above the Poverty Level     object
Median Income                                     object
Jobs Proximity Index                             float64
dtype: object

In [30]:
for col in df_merged.columns:
    if col != "2020 Census Tract":
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

        col_median = df_merged[col].median()
        
        new_col_name = f"above_median_{col}"
        
        df_merged[new_col_name] = np.where(df_merged[col] > col_median, 
                                    1, 
                                    0)


In [31]:
if not df_merged.empty:
    df_merged.to_csv('../../data/processed/scoring_indicators/stable_communities_2024_processed.csv', index=False)
else:
    print("Warning: df_merged is empty. CSV file not saved.")